In [ ]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
import glob
import random
from google.colab.patches import cv2_imshow




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
batchSize=2
imageSize=[600,600]
imgs=[]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')   # train on the GPU or on the CPU, if a GPU is not available
print(device)

imagePath = r"/content/drive/MyDrive/Colab Notebooks/FibreAnalysis"

sample_images = os.path.join(imagePath, 'Data', 'synth', 'images', '')
sample_masks = os.path.join(imagePath, 'Data', 'synth', 'masks', '')

file_pattern = os.path.join(sample_images, "*.png")

# Use glob to retrieve all matching file paths
file_paths = glob.glob(file_pattern)
# Extract filenames without the path
imgs = [os.path.basename(file_path) for file_path in file_paths]


In [ ]:


def loadData():
  batch_Imgs=[]
  batch_Data=[]
  for i in range(batchSize):
        idx=random.randint(0,len(imgs)-1)
        img = cv2.imread(os.path.join(sample_images , imgs[idx]))
        img = cv2.resize(img, imageSize, cv2.INTER_LINEAR)

        sample_masks = os.path.join(imagePath, 'Data', 'synth', 'masks', '')
        masks=[]

        filename, extension = os.path.splitext(imgs[idx])
        maskFile =  "mask" + filename[5:] + extension

        vesMask = cv2.imread(os.path.join(sample_masks , maskFile), 0)


        vesMask = (vesMask > 0).astype(np.uint8)
        vesMask=cv2.resize(vesMask,imageSize,cv2.INTER_NEAREST)

        masks.append(vesMask)
        num_objs = len(masks)

        if num_objs==0: return loadData()
        boxes = torch.zeros([num_objs,4], dtype=torch.float32)

        for i in range(num_objs):
            x,y,w,h = cv2.boundingRect(masks[i])
            boxes[i] = torch.tensor([x, y, x+w, y+h])

        masks = torch.as_tensor(masks, dtype=torch.uint8)

        #tempImg = img

        img = torch.as_tensor(img, dtype=torch.float32)
        data = {}
        data["boxes"] =  boxes
        data["labels"] =  torch.ones((num_objs,), dtype=torch.int64)
        data["masks"] = masks
        batch_Imgs.append(img)
        batch_Data.append(data)

        # Draw the bounding box on the image
        # cv2.rectangle(tempImg, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # # # Display the image
        # cv2.imshow("Image with Bounding Box", tempImg)

        # # # Wait for a key press and close the windows
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

  batch_Imgs=torch.stack([torch.as_tensor(d) for d in batch_Imgs],0)
  batch_Imgs = batch_Imgs.swapaxes(1, 3).swapaxes(2, 3)
  return batch_Imgs, batch_Data


In [ ]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)  # load an instance segmentation model pre-trained pre-trained on COCO
in_features = model.roi_heads.box_predictor.cls_score.in_features  # get number of input features for the classifier
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes=2)  # replace the pre-trained head with a new one
model.to(device)# move model to the right devic

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
model.train()

for i in range(10001):
            images, targets = loadData()
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            optimizer.zero_grad()
            loss_dict = model(images, targets)

            losses = sum(loss for loss in loss_dict.values())
            losses.backward()
            optimizer.step()
            print(i,'loss:', losses.item())
            if i%500==0:
                torch.save(model.state_dict(), str(i)+".torch")

0 loss: 140.54576110839844
1 loss: 104.70317077636719
2 loss: 91.79979705810547
3 loss: 54.086448669433594
4 loss: 54.354862213134766
5 loss: 40.868892669677734
6 loss: 80.87173461914062
7 loss: 22.352415084838867
8 loss: 23.416833877563477
9 loss: 13.695427894592285
10 loss: 30.184099197387695
11 loss: 38.690616607666016
12 loss: 19.792510986328125
13 loss: 16.03076171875
14 loss: 23.81993293762207
15 loss: 18.09040641784668
16 loss: 14.764626502990723
17 loss: 62.33615493774414
18 loss: 14.776785850524902
19 loss: 33.699256896972656
20 loss: 16.10779571533203
21 loss: 18.11185073852539
22 loss: 27.99099349975586
23 loss: 33.534732818603516
24 loss: 9.124326705932617
25 loss: 6.306549549102783
26 loss: 25.83587646484375
27 loss: 6.8132643699646
28 loss: 33.388797760009766
29 loss: 9.37600326538086
30 loss: 10.53812026977539
31 loss: 8.676900863647461
32 loss: 16.09988021850586
33 loss: 8.808692932128906
34 loss: 12.107831001281738
35 loss: 7.265483856201172
36 loss: 22.470596313476562

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model=torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor=FastRCNNPredictor(in_features,num_classes=2)
model.load_state_dict(torch.load("100.torch"))
model.to(device)# move model to the right devic
model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
# Get test image


test_images = os.path.join(imagePath, 'Data', 'synth', 'images', '')


actualTestImage = os.path.join(test_images, "image_2023-06-23_15-16-59-395777.png")

In [ ]:
images = cv2.imread(actualTestImage)
images = cv2.resize(images, imageSize, cv2.INTER_LINEAR)
image1 = images
images = torch.as_tensor(images, dtype=torch.float32).unsqueeze(0)
images=images.swapaxes(1, 3).swapaxes(2, 3)
images = list(image.to(device) for image in images)
with torch.no_grad():
    pred = model(images)


In [ ]:
pred

[{'boxes': tensor([], device='cuda:0', size=(0, 4)),
  'labels': tensor([], device='cuda:0', dtype=torch.int64),
  'scores': tensor([], device='cuda:0'),
  'masks': tensor([], device='cuda:0', size=(0, 1, 600, 600))}]

In [ ]:
# cv2_imshow(image1)
# cv2.waitKey()

In [ ]:
im= images[0].swapaxes(0, 2).swapaxes(0, 1).detach().cpu().numpy().astype(np.uint8)

im2 = im.copy()

for i in range(len(pred[0]['masks'])):
    msk=pred[0]['masks'][i,0].detach().cpu().numpy()
    scr=pred[0]['scores'][i].detach().cpu().numpy()
    if scr>0.8 :
        im2[:,:,0][msk>0.5] = random.randint(0,255)
        im2[:, :, 1][msk > 0.5] = random.randint(0,255)
        im2[:, :, 2][msk > 0.5] = random.randint(0, 255)

cv2.imshow(str(scr), np.hstack([im,im2]))
cv2.waitKey()

NameError: ignored